In [1]:
from utils import Settings
from openai import OpenAI

api_key = Settings().openai_api_key

#### podstawowe uzycie openai

In [3]:
from openai import OpenAI
client = OpenAI(api_key=api_key)

response = client.responses.create(
    model="gpt-5",
    input="Write a short bedtime story about a unicorn."
)

print(response.output_text)


In a valley where the grass hummed and the moon kept a quiet clock, there lived a unicorn named Liora. Her mane fell like rain on leaves, and her horn held a small, steady glow—no brighter than a candle, just enough to find the kindest path.

Each night, Liora walked the soft places: over moss pillows, beside sleeping ferns, along the edge of the dreaming stream. She had a gentle job, easier than it sounded: to gather the loose dreams that wandered from windows and guide them back to where they belonged.

One evening, she paused beneath a sill where a child sat awake, eyes like little lakes catching starlight.

“Can’t sleep?” Liora asked, in a voice like the hush between two heartbeats.

“The night feels too big,” the child whispered.

“Then we make it smaller,” said Liora, “breath by breath.”

She lowered her head, and the light at her horn unfurled into a thin silver path. In a blink that felt like a sigh, the child found themself in a meadow of velvet dark. Night-blooming flowers op

In [4]:
response

Response(id='resp_00e811964b869a4d00690c7f469134819cbea6db5fe6dd3110', created_at=1762426694.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-5-2025-08-07', object='response', output=[ResponseReasoningItem(id='rs_00e811964b869a4d00690c7f472ca8819c9f4cb5c2c7e24531', summary=[], type='reasoning', content=None, encrypted_content=None, status=None), ResponseOutputMessage(id='msg_00e811964b869a4d00690c7f5ffe2c819cb295339d62602a84', content=[ResponseOutputText(annotations=[], text='In a valley where the grass hummed and the moon kept a quiet clock, there lived a unicorn named Liora. Her mane fell like rain on leaves, and her horn held a small, steady glow—no brighter than a candle, just enough to find the kindest path.\n\nEach night, Liora walked the soft places: over moss pillows, beside sleeping ferns, along the edge of the dreaming stream. She had a gentle job, easier than it sounded: to gather the loose dreams that wandered from windows and guide them ba

#### structured output
https://platform.openai.com/docs/guides/structured-outputs

In [6]:
with open('data/emails/5e0d726e-4ff1-47f5-b02a-84e060aea8fe.txt', encoding='utf-8') as file:
    content = file.read()

In [7]:
content

'Mogę podesłać zdjęcia lub nagranie, jeśli będzie potrzebne. Adres można zapisać także jako 57-554 Katowice, Krótka nr 114. Może to nie ma znaczenia, ale problem zaczyna się zwykle wieczorem, a rano jest spokojniej. Mieszkańcy pod adresem Katowice, ul. Krótka 114, kod 57-554, zgłaszają podobne trudności. Opis problemu: Nieszczelne okno w salonie, silne przewiewy.. Od kiedy pojawił się ten problem, rachunki są wyraźnie wyższe. Nie jestem pewien, czy numer domu to właściwie ten sam, bo na skrzynce pocztowej widnieje inny. Kolega z pracy powiedział, że miał taką samą awarię i naprawa trwała tydzień. Adres zgłoszenia to Krótka 114, 57-554 Katowice. W dokumentach widnieje numer umowy UM-1044, który był już wcześniej podawany. Sprawa dotyczy również bezpieczeństwa mieszkańców. W weekend było gorzej niż w dni powszednie, co też warto odnotować. Zgłoszenie dotyczy umowy nr UM-1044, sprawa związana z awarią: Nieszczelne okno w salonie, silne przewiewy.. Numer kontaktowy zostawiłem już w poprzed

In [ ]:
client = OpenAI()

response = client.responses.create(
    model="gpt-4o-2024-08-06",
    input=[
        {"role": "system", "content": '''Z podanego tekstu wyciagnij informacje o numerze umowy, którego dotyczy zdarzenie, krótki opis zdarzenia, dane adresowe w formie - 
         nazwa miasta, nazwa ulicy, numer domu i kod pocztowy. W odpowiedzi zamieść także nazwę pliku.'''},
        {"role": "user", "content": content},
    ],
    temperature=0
)

In [24]:
print(response.output[0].content[0].text)

**Nazwa pliku**: zgłoszenie_awarii_UM-1044.txt

- **Adres**: Katowice, ul. Krótka 114, 57-554
- **Numer umowy**: UM-1044
- **Opis zdarzenia**: Nieszczelne okno w salonie, silne przewiewy.


In [58]:
from pydantic import BaseModel
from typing import Optional
from pprint import pprint

In [48]:
# {
#     "contract_number": "sdasd",
#     "short_desc": "asdasd",
#     "filename": "asdasd.txt"
#     "address": {
#         "city": "asdasd",
#         "postal_code":
#         "street_name": 
#         "house_number"
#     }
# }

class Address(BaseModel):
    city: Optional[str] = None
    postal_code: Optional[str] = None
    street_name: Optional[str] = None
    house_number: Optional[str] = None


class Issue(BaseModel):
    contract_number: Optional[str] = None
    short_desc: Optional[str] = None
    address: Optional[Address] = None

PROMPT = 'Uzyskaj z tekstu informacje o numerze umowy, którego dotyczy zgłoszenie, skórcony opis zdarzenia oraz adres.'

In [29]:
with open('data/emails/5e0d726e-4ff1-47f5-b02a-84e060aea8fe.txt', encoding='utf-8') as file:
    content = file.read()

In [49]:
client = OpenAI()

response = client.responses.parse(
    model="gpt-4o-2024-08-06",
    input=[
        {"role": "system", "content": PROMPT},
        {"role": "user", "content": content},
    ],
    text_format=Issue,
    temperature=0
)

In [50]:
issue = response.output_parsed

In [61]:
response

ParsedResponse[Issue](id='resp_082a483bf0a8143d00690c85f900cc819c9601758fed9b7683', created_at=1762428409.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4o-2024-08-06', object='response', output=[ParsedResponseOutputMessage[Issue](id='msg_082a483bf0a8143d00690c85f98860819cb27d3af04a25e660', content=[ParsedResponseOutputText[Issue](annotations=[], text='{"contract_number":"UM-1044","short_desc":"Nieszczelne okno w salonie, silne przewiewy.","address":{"city":"Katowice","postal_code":"57-554","street_name":"Krótka","house_number":"114"}}', type='output_text', logprobs=[], parsed=Issue(contract_number='UM-1044', short_desc='Nieszczelne okno w salonie, silne przewiewy.', address=Address(city='Katowice', postal_code='57-554', street_name='Krótka', house_number='114')))], role='assistant', status='completed', type='message')], parallel_tool_calls=True, temperature=0.0, tool_choice='auto', tools=[], top_p=1.0, background=False, conversation=None, max_output

In [51]:
issue.address.postal_code

'57-554'

In [59]:
pprint(issue.model_dump()) # automatyczna konwersja modelu danych na obiekt python
issue.model_dump_json() # automatyczna konwersja modelu danych na json

{'address': {'city': 'Katowice',
             'house_number': '114',
             'postal_code': '57-554',
             'street_name': 'Krótka'},
 'contract_number': 'UM-1044',
 'short_desc': 'Nieszczelne okno w salonie, silne przewiewy.'}


'{"contract_number":"UM-1044","short_desc":"Nieszczelne okno w salonie, silne przewiewy.","address":{"city":"Katowice","postal_code":"57-554","street_name":"Krótka","house_number":"114"}}'

#### zadanie openai structured output

In [ ]:
# z wykorzystaniem powyższej implementacji napisz pipeline do przetworzenia wielu plików tekstowych
# wynik zapis do ramki danych pandas DataFrame
# dane w folderze data/emails

In [ ]:
# {
#     "bbox": [(x1, y1), (x1, y2), (x2,y1), (x2,y2)],
#     "obj_type": "spare cable"
# }

{
    "bbox": {
        'p1': {"x": 12, "y": 45}
    },
    "obj_type": "spare cable"
}